### Next work prediction using LSTM

In [2]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## Load dataset
data = gutenberg.raw('shakespeare-hamlet.txt')


with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\hsingh\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [9]:
## Data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## Load dataset
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

## Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [10]:
## Create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences[:10]

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891]]

In [13]:
## Pad sequences
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))
input_sequences[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    1,  687],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,  687,    4],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,
         687,    4,   45],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    1,  687,
           4,   45,   41],
       [   0,    0,    0,    0,    0,    0,    0,    0,    1,  687,    4,
          45,   41, 1886],
       [   0,    0,    0,    0,    0,    0,    0,    1,  687,    4,   45,
          41, 1886, 1887],
       [   0,    0,    0,    0,    0,    0,    1,  687,    4,   45,   41,
        1886, 1887, 1888],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 1180, 1889],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        1180, 1889, 1890],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1180,
        1889, 189

In [14]:
## Create predictors and label
import tensorflow as tf
x,y = input_sequences[:,:-1], input_sequences[:,-1]
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [15]:
y

array([ 687,    4,   45, ..., 1047,    4,  193])

In [17]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
## Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
## Early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [21]:
## Train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

## Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
## Train model
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), verbose=1)

Epoch 1/50


644/644 [==============================] - 45s 58ms/step - loss: 6.9132 - accuracy: 0.0333 - val_loss: 6.6976 - val_accuracy: 0.0356
Epoch 2/50
644/644 [==============================] - 18s 28ms/step - loss: 6.4775 - accuracy: 0.0390 - val_loss: 6.7558 - val_accuracy: 0.0414
Epoch 3/50
644/644 [==============================] - 18s 28ms/step - loss: 6.3288 - accuracy: 0.0472 - val_loss: 6.7969 - val_accuracy: 0.0482
Epoch 4/50
644/644 [==============================] - 17s 27ms/step - loss: 6.1845 - accuracy: 0.0522 - val_loss: 6.8229 - val_accuracy: 0.0488
Epoch 5/50
644/644 [==============================] - 19s 29ms/step - loss: 6.0427 - accuracy: 0.0571 - val_loss: 6.8512 - val_accuracy: 0.0567
Epoch 6/50
644/644 [==============================] - 20s 31ms/step - loss: 5.8956 - accuracy: 0.0634 - val_loss: 6.9263 - val_accuracy: 0.0571
Epoch 7/50
644/644 [==============================] - 24s 37ms/step - loss: 5.7534 - accuracy: 0.0721 - val_loss: 6.9412 - val_accurac

In [23]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [28]:
input_text = "Mar. Thus twice before, and iust at this dead"
print(f"Input test: {input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, text=input_text, max_sequence_len=max_sequence_len)
print(f"Next word: {next_word}")

Input test: Mar. Thus twice before, and iust at this dead
Next word: houre


In [29]:
## Save the model
model.save('next_word_lstm.h5')

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\hsingh\OneDrive - INVIDI Technologies Corp\machine learning\machine-learning\myenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
